# Users Web Scrapping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time 
import csv 
from random import randint

In [2]:
def get_value(text):
    # Función para conseguir el texto entre los markups
    text = str(text)
    text = text.replace("\n", "").replace("\r", "").replace("\t", "")
    markup_count = text.count("</")
    position = -1
    for i in range(markup_count):
        position = text.find(">", position + 1)
    first_close = text.find("</")
    return text[position + 1: first_close]
    
def get_rating_value(text):
    # Función para conseguir el rating con el que se califica un anime
    text = str(text)
    text = text.replace("\n", "").replace("\r", "").replace("\t", "").replace(" ", "")
    rating_text = text.split(':')[-1]
    return rating_text.split('<')[0]

def get_anime_rating(text):
    # Función para conseguir un anime y su rating desde la lista de anime list de un usuario
    splited_obj = text.split(',')
    if len(splited_obj) == 28:
        anime_id = splited_obj[7].split('"anime_id":')[1]
        anime = splited_obj[4].split('"anime_title":"')[1][:-1]
        rating = splited_obj[0].split(':')[1]
        return (anime_id, anime, rating)
    return (None, None, None)
    
    
    

In [3]:
def product_male_user_list_data(total_data = 50000):
    '''
    Scrapping para obtener la calificación de los usuarios a los animé que han visto.
    Recibe como parámetro total_data, que corresponde al total de filas que se quiere obtener.
    '''

    base_url = 'https://myanimelist.net/'
    url = 'https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028'
    actual_url = url
    users_information = [[], []] # [ [username], [user gender]]
    ratings_information = [[], [], []] # [ [username], [anime], [overall rating] ]
    
    while True:
        print('actual url: ', actual_url)
        main_page = requests.get(actual_url)
        soup = BeautifulSoup(main_page.content, 'html.parser')
        tables = soup.find_all('table')
        user_rows = tables[1].find_all('tr')
    
        for row in user_rows:
            user_icons = row.find_all('td')
            for icon in user_icons:
                
                user_name = icon.find('div').find('a')
                user_name = get_value(user_name)
                users_information[0].append(user_name)
                
                user_href = icon.find('div').find('a', href=True)
                user_url = base_url + user_href['href']
                
                # Ingresamos al perfil del usuario para obtener género y anime list
                time.sleep(randint(1,3))
                user_page = requests.get(user_url)
                user_soup = BeautifulSoup(user_page.content, 'html.parser')
                user_info = user_soup.find('ul', {'class': 'user-status'})
                if user_info is None:
                    gender = 'Not Found'
                else:
                    span = user_info.find_all('li')[1].find_all('span')
                    if get_value(span[0]) == 'Gender':
                        gender = get_value(span[1])
                    else:
                        gender = 'None'
                users_information[1].append(gender)
                
                # Escribimos los datos del usuario en el csv
                with open('male_users.csv', mode='a') as users_file:
                    users_writer = csv.writer(users_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    users_writer.writerow([user_name, gender]) 
    
                user_lists = user_soup.find('div', {'class': 'user-button clearfix mb12'})
                user_anime_list = user_lists.find('a', href=True)
                user_anime_list_url = user_anime_list['href']
                
                # Ingresamos al anime list del usuario
                anime_list = requests.get(user_anime_list_url)
                anime_list_soup = BeautifulSoup(anime_list.content, 'html.parser')
                status_menu = anime_list_soup.find('div', {'class': 'status-menu-container'})
                if status_menu:
                    try:
                        completed_anime_list_url = status_menu.find('a', {'class': 'status-button completed'}, href=True)['href']

                        # Ingresamos a anime list completos para extraer su calificacion
                        completed_anime_list = requests.get(completed_anime_list_url)
                        completed_anime_list_soup = BeautifulSoup(completed_anime_list.content, 'html.parser')
                        completed_list = completed_anime_list_soup.find('div', {'class': 'list-unit completed'})

                        objects_completed_list = completed_list.find('table')['data-items']
                        objects_completed_list = objects_completed_list.split('"status":2,')

                        # Tomamos los datos de los distintos anime y guardamos el rating
                        for _object in objects_completed_list:
                            anime_id, anime, rating = get_anime_rating(_object)
                            if anime and rating:
                                ratings_information[0].append(user_name)
                                ratings_information[1].append(anime)
                                ratings_information[2].append(rating)
                                with open('male_ratings.csv', mode='a') as ratings_file:
                                    ratings_writer = csv.writer(ratings_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    ratings_writer.writerow([user_name, anime_id, anime, rating]) 

                        # Revisamos si conseguimos el total de usuarios que queremos
                        print("users:info[0]: ", len(users_information[0]), " total_data: ", total_data)
                        if len(users_information[0]) == total_data:
                            print("break")
                            break
                    
                    except:
                        pass
                    

            # Revisamos si conseguimos el total de usuarios que queremos
            print("users:info[0]: ", len(users_information[0]), " total_data: ", total_data)
            if len(users_information[0]) == total_data:
                print("break")
                break
                
         # Revisamos si conseguimos el total de usuarios que queremos
        print("users:info[0]: ", len(users_information[0]), " total_data: ", total_data)
        if len(users_information[0]) == total_data:
            print("break")
            break
        
        #Si aún no hemos conseguido la cantidad de datos buscada, cambiamos de url a la siguiente página y realizamos nuevamente el proceso.
        if actual_url == 'https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1': 
            next_page = url + '&show=24'
        else:
            next_page = '&show=' + str(int(actual_url.split('=')[-1]) + 24)
        actual_url = url + next_page
  
        
    return 
     

In [4]:
product_male_user_list_data()

actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028
users:info[0]:  2  total_data:  50000
users:info[0]:  3  total_data:  50000
users:info[0]:  4  total_data:  50000
users:info[0]:  6  total_data:  50000
users:info[0]:  6  total_data:  50000
users:info[0]:  7  total_data:  50000
users:info[0]:  8  total_data:  50000
users:info[0]:  9  total_data:  50000
users:info[0]:  10  total_data:  50000
users:info[0]:  11  total_data:  50000
users:info[0]:  12  total_data:  50000
users:info[0]:  12  total_data:  50000
users:info[0]:  13  total_data:  50000
users:info[0]:  14  total_data:  50000
users:info[0]:  17  total_data:  50000
users:info[0]:  18  total_data:  50000
users:info[0]:  20  total_data:  50000
users:info[0]:  21  total_data:  50000
users:info[0]:  23  total_data:  50000
users:info[0]:  24  total_data:  50000
users:info[0]:  24  total_data:  50000
users:info[0]:  24  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=us

users:info[0]:  187  total_data:  50000
users:info[0]:  188  total_data:  50000
users:info[0]:  189  total_data:  50000
users:info[0]:  190  total_data:  50000
users:info[0]:  191  total_data:  50000
users:info[0]:  192  total_data:  50000
users:info[0]:  192  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028&show=6220
users:info[0]:  193  total_data:  50000
users:info[0]:  195  total_data:  50000
users:info[0]:  197  total_data:  50000
users:info[0]:  198  total_data:  50000
users:info[0]:  198  total_data:  50000
users:info[0]:  199  total_data:  50000
users:info[0]:  200  total_data:  50000
users:info[0]:  201  total_data:  50000
users:info[0]:  202  total_data:  50000
users:info[0]:  203  total_data:  50000
users:info[0]:  204  total_data:  50000
users:info[0]:  204  total_data:  50000
users:info[0]:  205  total_data:  50000
users:info[0]:  206  total_data:  50000
users:info[0]:  207  total_data:  50000
users:info[0]

users:info[0]:  368  total_data:  50000
users:info[0]:  369  total_data:  50000
users:info[0]:  370  total_data:  50000
users:info[0]:  371  total_data:  50000
users:info[0]:  372  total_data:  50000
users:info[0]:  372  total_data:  50000
users:info[0]:  373  total_data:  50000
users:info[0]:  374  total_data:  50000
users:info[0]:  375  total_data:  50000
users:info[0]:  376  total_data:  50000
users:info[0]:  377  total_data:  50000
users:info[0]:  378  total_data:  50000
users:info[0]:  378  total_data:  50000
users:info[0]:  379  total_data:  50000
users:info[0]:  380  total_data:  50000
users:info[0]:  381  total_data:  50000
users:info[0]:  382  total_data:  50000
users:info[0]:  383  total_data:  50000
users:info[0]:  384  total_data:  50000
users:info[0]:  384  total_data:  50000
users:info[0]:  384  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028&show=6412
users:info[0]:  385  total_data:  50000
users:info[0]

users:info[0]:  549  total_data:  50000
users:info[0]:  550  total_data:  50000
users:info[0]:  551  total_data:  50000
users:info[0]:  552  total_data:  50000
users:info[0]:  552  total_data:  50000
users:info[0]:  552  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028&show=6580
users:info[0]:  554  total_data:  50000
users:info[0]:  557  total_data:  50000
users:info[0]:  558  total_data:  50000
users:info[0]:  558  total_data:  50000
users:info[0]:  559  total_data:  50000
users:info[0]:  560  total_data:  50000
users:info[0]:  561  total_data:  50000
users:info[0]:  562  total_data:  50000
users:info[0]:  563  total_data:  50000
users:info[0]:  564  total_data:  50000
users:info[0]:  564  total_data:  50000
users:info[0]:  565  total_data:  50000
users:info[0]:  566  total_data:  50000
users:info[0]:  567  total_data:  50000
users:info[0]:  568  total_data:  50000
users:info[0]:  569  total_data:  50000
users:info[0]

users:info[0]:  721  total_data:  50000
users:info[0]:  722  total_data:  50000
users:info[0]:  723  total_data:  50000
users:info[0]:  724  total_data:  50000
users:info[0]:  725  total_data:  50000
users:info[0]:  726  total_data:  50000
users:info[0]:  726  total_data:  50000
users:info[0]:  727  total_data:  50000
users:info[0]:  728  total_data:  50000
users:info[0]:  729  total_data:  50000
users:info[0]:  732  total_data:  50000
users:info[0]:  733  total_data:  50000
users:info[0]:  734  total_data:  50000
users:info[0]:  735  total_data:  50000
users:info[0]:  736  total_data:  50000
users:info[0]:  737  total_data:  50000
users:info[0]:  738  total_data:  50000
users:info[0]:  738  total_data:  50000
users:info[0]:  739  total_data:  50000
users:info[0]:  740  total_data:  50000
users:info[0]:  742  total_data:  50000
users:info[0]:  743  total_data:  50000
users:info[0]:  744  total_data:  50000
users:info[0]:  744  total_data:  50000
users:info[0]:  744  total_data:  50000


...

users:info[0]:  5914  total_data:  50000
users:info[0]:  5915  total_data:  50000
users:info[0]:  5916  total_data:  50000
users:info[0]:  5916  total_data:  50000
users:info[0]:  5917  total_data:  50000
users:info[0]:  5919  total_data:  50000
users:info[0]:  5921  total_data:  50000
users:info[0]:  5922  total_data:  50000
users:info[0]:  5922  total_data:  50000
users:info[0]:  5923  total_data:  50000
users:info[0]:  5924  total_data:  50000
users:info[0]:  5925  total_data:  50000
users:info[0]:  5926  total_data:  50000
users:info[0]:  5927  total_data:  50000
users:info[0]:  5928  total_data:  50000
users:info[0]:  5928  total_data:  50000
users:info[0]:  5928  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028&show=11956
users:info[0]:  5929  total_data:  50000
users:info[0]:  5930  total_data:  50000
users:info[0]:  5931  total_data:  50000
users:info[0]:  5933  total_data:  50000
users:info[0]:  5934  total_dat

users:info[0]:  6079  total_data:  50000
users:info[0]:  6080  total_data:  50000
users:info[0]:  6082  total_data:  50000
users:info[0]:  6083  total_data:  50000
users:info[0]:  6084  total_data:  50000
users:info[0]:  6085  total_data:  50000
users:info[0]:  6086  total_data:  50000
users:info[0]:  6087  total_data:  50000
users:info[0]:  6088  total_data:  50000
users:info[0]:  6089  total_data:  50000
users:info[0]:  6090  total_data:  50000
users:info[0]:  6090  total_data:  50000
users:info[0]:  6091  total_data:  50000
users:info[0]:  6092  total_data:  50000
users:info[0]:  6093  total_data:  50000
users:info[0]:  6094  total_data:  50000
users:info[0]:  6095  total_data:  50000
users:info[0]:  6096  total_data:  50000
users:info[0]:  6096  total_data:  50000
users:info[0]:  6096  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028&show=12124
users:info[0]:  6097  total_data:  50000
users:info[0]:  6098  total_dat

users:info[0]:  6249  total_data:  50000
users:info[0]:  6250  total_data:  50000
users:info[0]:  6251  total_data:  50000
users:info[0]:  6252  total_data:  50000
users:info[0]:  6253  total_data:  50000
users:info[0]:  6254  total_data:  50000
users:info[0]:  6255  total_data:  50000
users:info[0]:  6256  total_data:  50000
users:info[0]:  6257  total_data:  50000
users:info[0]:  6258  total_data:  50000
users:info[0]:  6258  total_data:  50000
users:info[0]:  6259  total_data:  50000
users:info[0]:  6260  total_data:  50000
users:info[0]:  6261  total_data:  50000
users:info[0]:  6262  total_data:  50000
users:info[0]:  6264  total_data:  50000
users:info[0]:  6264  total_data:  50000
users:info[0]:  6264  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=1&show=6028&show=12292
users:info[0]:  6265  total_data:  50000
users:info[0]:  6266  total_data:  50000
users:info[0]:  6267  total_data:  50000
users:info[0]:  6268  total_dat

users:info[0]:  6409  total_data:  50000
users:info[0]:  6410  total_data:  50000
users:info[0]:  6411  total_data:  50000
users:info[0]:  6412  total_data:  50000
users:info[0]:  6413  total_data:  50000
users:info[0]:  6414  total_data:  50000
users:info[0]:  6414  total_data:  50000
users:info[0]:  6415  total_data:  50000
users:info[0]:  6416  total_data:  50000
users:info[0]:  6418  total_data:  50000
users:info[0]:  6419  total_data:  50000
users:info[0]:  6420  total_data:  50000
users:info[0]:  6420  total_data:  50000
users:info[0]:  6422  total_data:  50000
users:info[0]:  6426  total_data:  50000
users:info[0]:  6426  total_data:  50000
users:info[0]:  6427  total_data:  50000
users:info[0]:  6428  total_data:  50000
users:info[0]:  6429  total_data:  50000
users:info[0]:  6430  total_data:  50000
users:info[0]:  6431  total_data:  50000
users:info[0]:  6432  total_data:  50000
users:info[0]:  6432  total_data:  50000
users:info[0]:  6432  total_data:  50000
actual url:  htt

KeyboardInterrupt: 